In [1]:
%matplotlib notebook
from util import printColAssets, printSummary, User, get_truncated_normal
import matplotlib.pyplot as plt
import multiprocessing as mp
from simulation_util import *
import sys
import numpy as np
from collections import defaultdict
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Plotter Functions

In [2]:
def plot3D(x, y, z, xtitle, ytitle, ztitle):
    fig = go.Figure(data=[
            go.Scatter3d(x=x, 
                         y=y, 
                         z=z,
                        line=dict(
                            color='darkblue',
                            width=2
                        ) 
                    )])
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
def processData(x,y,z):
    # Get x,y,z sorted based on y i.e TXF
    sortxyz = [(x,y,z) for y,x,z in sorted(zip(y,x,z))]
    
    x = [i[0] for i in sortxyz]
    y = [i[1] for i in sortxyz]
    z = [i[2] for i in sortxyz]
    
    return x,y,z
    
    
def plot3DWithTXF(x,y,z,xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    # Get x,y,z sorted based on y i.e TXF
    x,y,z = processData(x,y,z)
    
    txDictx = defaultdict(list)
    txDictz = defaultdict(list)
    
    for i in range(len(y)):
        txDictx[y[i]].append(x[i])
        txDictz[y[i]].append(z[i])
        
    data = []   
 
              
    cnt = 0          
    for key in txDictx.keys():
        
        x_d = txDictx[key]
        y_d = [key for i in range(len(txDictx[key]))]
        z_d = txDictz[key]

        temp = go.Scatter3d(x=x_d, 
                         y=y_d, 
                         z=z_d,
                        line=dict(
                            color=colors[cnt % len(colors)],
                            width=2
                        ) 
                    )
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
    
def plot2D(x, y, xtitle, ytitle):    
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers'))
    
    fig.update_layout(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
    )
    
    plotly.offline.iplot(fig)


# Simulation Functions

In [3]:
def runOnThread(sample_size, assets, risk_params, cdpRate, txFee, tests, stddev=False):
    avgMarketDAI = 0
    avgDAIPrice = 0

    history = []

    for i in range(tests):
        s = Simulator(rho=2.5, cdpRate=cdpRate, txf=txFee, eth_price=140, sample_size=sample_size,
                      initial_distribution=assets, risk_params=risk_params)
        a, b = s.runSimulation()
        avgDAIPrice += a
        avgMarketDAI += b

        if stddev:
            history.append(a)

    avgDAIPrice /= tests
    avgMarketDAI /= tests

    if stddev:
        print(" CDP Rate:" , cdpRate, "TxFee: ", txFee)
        print("Average: ", avgDAIPrice)
        print("stddev: ", np.std(np.array(history), dtype=np.float64))

    return avgDAIPrice

In [8]:
def runTests(sample_size, cdpRates, tcf, tests):
    
    # Get number of CPUs
    cpus = mp.cpu_count()
    poolCount = cpus * 2

    # Process pool to handle k simulations/process!
    pool = mp.Pool(processes=poolCount)

    args = []
    
    # Define initial allocation and risk distribution
    assets = getAssets(sample_size)
    risk_params = getRiskParams(sample_size)

    for txFee in txf:
        for cdpRate in cdpRates:
            args.append((sample_size, assets, risk_params, cdpRate, txFee, tests))

    results = pool.starmap(runOnThread, args)

    # Plot results here
    cdp_axis = [args[i][3] for i in range(len(args))]
    txf_axis = [args[i][4] for i in range(len(args))]
    dai_axis = results
    
    print("Now plotting ....")
    plot3DWithTXF(cdp_axis, txf_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")



# Transaction Fee varied, CDP Rate Varied

In [6]:
cdpRates = [0.01 * i for i in range(1, 35)]
txf = [0.01 * i for i in range(0, 15)]
tests = 1

runTests(10, cdpRates, txf, tests)

[(10, [[253.4828119306677, 1.6546471554380369, 714.9659080150104, 0], [528.3610972600356, 1.7113126520516666, 418.5274272368552, 0], [711.9572464419152, 1.1966422002196195, 755.2452979474326, 0], [681.3297120808013, 1.2075394800189079, 685.951472739838, 0], [573.8818348944824, 0.889640371041177, 255.98464347708438, 0], [366.23520759162597, 1.2552416262212651, 804.7927260644624, 0], [560.4736180703322, 1.3849986052052445, 590.1815440424155, 0], [961.7003841058065, 2.3262030463191596, 550.1979501706361, 0], [685.8111838902466, 1.6630951884825111, 765.1612859132324, 0], [661.2298725877101, 1.6764192408540852, 564.6740578315283, 0]], [0.001157551610437916, 0.0013278045533251194, 0.0026801144399054614, 0.005999175715893597, 0.006670362577007657, 0.008401203621293213, 0.009066331388552317, 0.009622569596966501, 0.010143886288926617, 0.01080384750862058], 0.01, 0.0, 1), (10, [[253.4828119306677, 1.6546471554380369, 714.9659080150104, 0], [528.3610972600356, 1.7113126520516666, 418.52742723685

164
167
167
172
179
178
179
178
181
202
202
206
206
214
219
223
223
226
230
239
247
247
276
276
168
168
189
189
206
206
207
183
198
227
202
202
201
193
253
207
229
210
229
238
226
254
226
254
172
172
181
219
219
164
164
167
206
206
178
179
230
214
206
206
276
239
223
202
226
226
247
247
207
207
168
207
198
227
207
198
206
189
253
253
183
201
183
238
210
202
226
229
193
193
181
181
167
172
206
219
254
254
230
230
239
239
179
214
164
214
202
178
178
223
206
276
276
247
206
227
227
207
168
198
207
226
210
226
210
201
201
238
238
202
189
189
183
253
164
172
229
226
226
206
181
219
254
206
193
226
193
167
178
214
230
179
172
202
239
164
223
276
207
206
247
198
219
253
183
207
206
181
227
193
189
206
168
226
201
207
202
238
210
181
198
229
226
253
207
167
183
230
178
172
254
214
179
227
206
193
239
164
219
181
276
202
223
227
206
230
214
206
247
189
207
168
167
201
178
239
238
226
198
210
276
207
227
226
253
202
167
183
172
229
201
181
179
206
189
206
238
219
193
210
202
254
167
230
164
223


# Tests to check price settling variance

In [10]:
def runOnThreadVariance(sample_size, cdpRate, txFee, tests):
    avgDAIPrice = 0

    history = []

    for i in range(tests):
        assets = getAssets(sample_size)
        risk_params = getRiskParams(sample_size)
        s = Simulator(rho=2.5, cdpRate=cdpRate, txf=txFee, eth_price=140, sample_size=sample_size,
                      initial_distribution=assets, risk_params=risk_params)
        a, _ = s.runSimulation()
        avgDAIPrice += a
        history.append(a)

    avgDAIPrice /= tests

    return avgDAIPrice, np.std(np.array(history), dtype=np.float64)

def runTestsVariance(sample_size, cdpRates, tcf, tests):
    
    # Get number of CPUs
    cpus = mp.cpu_count()
    poolCount = cpus * 2

    # Process pool to handle k simulations/process!
    pool = mp.Pool(processes=poolCount)

    args = []

    for txFee in txf:
        for cdpRate in cdpRates:
            args.append((sample_size, cdpRate, txFee, tests))

    results = pool.starmap(runOnThreadVariance, args)

    # Plot results here
    cdp_axis = [args[i][1] for i in range(len(args))]
    txf_axis = [args[i][2] for i in range(len(args))]
 
    return cdp_axis, txf_axis, results

In [19]:
cdpRates = [0.01 * i for i in range(1, 10)]
txf = [0.01 * i for i in range(0, 10)]
tests = 10

cdp_axis, txf_axis, results = runTestsVariance(10, cdpRates, txf, tests)

for i in range(len(results)):
    print("CDP Rate:" , cdp_axis[i], "TxFee: ", txf_axis[i])
    print("Average: ", results[i][0])
    print("stddev: ", results[i][1])
    print()

 CDP Rate: 0.01 TxFee:  0.0
Average:  1.9666999999999846
stddev:  0.10723530202316513
 CDP Rate: 0.02 TxFee:  0.0
Average:  2.0344999999999795
stddev:  0.10557959083079599
 CDP Rate: 0.03 TxFee:  0.0
Average:  2.0416999999999783
stddev:  0.10058931354770384
 CDP Rate: 0.04 TxFee:  0.0
Average:  2.0179999999999803
stddev:  0.11583004791502911
 CDP Rate: 0.05 TxFee:  0.0
Average:  2.057199999999976
stddev:  0.12679416390353912
 CDP Rate: 0.06 TxFee:  0.0
Average:  2.0988999999999725
stddev:  0.1193125726820013
 CDP Rate: 0.07 TxFee:  0.0
Average:  2.0744999999999765
stddev:  0.08152330955008273
 CDP Rate: 0.08 TxFee:  0.0
Average:  2.0584999999999773
stddev:  0.08990912078314611
 CDP Rate: 0.09 TxFee:  0.0
Average:  2.046999999999978
stddev:  0.10362432146942308
 CDP Rate: 0.01 TxFee:  0.01
Average:  1.9666999999999846
stddev:  0.10723530202316513
 CDP Rate: 0.02 TxFee:  0.01
Average:  2.0344999999999795
stddev:  0.10557959083079599
 CDP Rate: 0.03 TxFee:  0.01
Average:  2.04169999999997

TypeError: 'float' object is not subscriptable

### Standard Deviation across multiple tests seems to lie withing $0.21


# Test with initial assset distribution as 100 USD or 100 DAI

In [9]:
cdpRates = [0.01 * i for i in range(1, 35)]
txf = [0.01 * i for i in range(0, 15)]
tests = 1

runTests(10, cdpRates, txf, tests)

Now plotting ....


# Variance Test with 100 USD 100 DAI

In [13]:
cdpRates = [0.01 * i for i in range(1, 10)]
txf = [0.01 * i for i in range(0, 10)]
tests = 10

cdp_axis, txf_axis, results = runTestsVariance(10, cdpRates, txf, tests)

for i in range(len(results)):
    print("CDP Rate:" , cdp_axis[i], "TxFee: ", txf_axis[i])
    print("Average: ", results[i][0])
    print("stddev: ", results[i][1])
    print()

CDP Rate: 0.01 TxFee:  0.0
Average:  0.9904999999999999
stddev:  0.0015000000000000013

CDP Rate: 0.02 TxFee:  0.0
Average:  0.99
stddev:  0.0

CDP Rate: 0.03 TxFee:  0.0
Average:  0.99
stddev:  0.0

CDP Rate: 0.04 TxFee:  0.0
Average:  0.99
stddev:  0.0

CDP Rate: 0.05 TxFee:  0.0
Average:  0.99
stddev:  0.0

CDP Rate: 0.06 TxFee:  0.0
Average:  0.9813000000000001
stddev:  0.017412926233117756

CDP Rate: 0.07 TxFee:  0.0
Average:  0.9765
stddev:  0.0206215906273013

CDP Rate: 0.08 TxFee:  0.0
Average:  0.9499000000000001
stddev:  0.013442098050527691

CDP Rate: 0.09 TxFee:  0.0
Average:  0.9510000000000002
stddev:  0.013076696830622032

CDP Rate: 0.01 TxFee:  0.01
Average:  0.9904999999999999
stddev:  0.0015000000000000013

CDP Rate: 0.02 TxFee:  0.01
Average:  0.99
stddev:  0.0

CDP Rate: 0.03 TxFee:  0.01
Average:  0.99
stddev:  0.0

CDP Rate: 0.04 TxFee:  0.01
Average:  0.99
stddev:  0.0

CDP Rate: 0.05 TxFee:  0.01
Average:  0.99
stddev:  0.0

CDP Rate: 0.06 TxFee:  0.01
Average: 

### Standard Dev across tests is under 0.02